## Coursera Capstone Project - Week 5(Final)
### Author: Himanshu Soni


   **About the project**
    
1. The following script helps potential clients who are looking to buy suitable property in London
2. With a pre-determined budged, this automation will recommend you locations and current average price of real estate where one can make a real estate investment
3. Important amenities/ venues for each recommended location are displayed
4. Facilities like elementary schools, high schools, hospitals & grocery stores are displayed for each recommended location

In [2]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('All Libraries are imported.')

Solving environment: ...working... 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.1.1a-he774522_0, defaults::certifi-2018.11.29-py37_0
  - anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-he774522_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::openssl-1.1.1a-he774522_0, defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0
  - anaconda::certifi-2018.11.29-py37_0, defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.1.1a-he774522_0
  - defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0done

## Package Plan ##

  e

'ET' is not recognized as an internal or external command,
operable program or batch file.


Solving environment: ...working... 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.1.1a-he774522_0, defaults::certifi-2018.11.29-py37_0
  - anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-he774522_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::openssl-1.1.1a-he774522_0, defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0
  - anaconda::certifi-2018.11.29-py37_0, defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.1.1a-he774522_0
  - defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0done

## Package Plan ##

  e

**We have to download source dataset file from following link :  http://landregistry.data.gov.uk/**

In [10]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("price-paid-data-monthly-update.csv")

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [11]:
print(df_ppd.shape)
df_ppd.head()

(89666, 16)


,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{8355F008-E832-55C5-E053-6B04A8C0D090},41000,2003-11-28 00:00,TN38 0JT,F,N,L,47,NaN,CARISBROOKE ROAD,NaN,ST LEONARDS-ON-SEA,HASTINGS,EAST SUSSEX,A,A
1,{8355F008-A9E2-55C5-E053-6B04A8C0D090},415000,2003-11-24 00:00,LU7 3AG,D,N,F,239,NaN,HEATH ROAD,NaN,LEIGHTON BUZZARD,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
2,{8355F009-06FF-55C5-E053-6B04A8C0D090},615000,2003-05-01 00:00,SO41 0HU,D,N,F,YEATTON FARM,NaN,HORDLE LANE,HORDLE,LYMINGTON,NEW FOREST,HAMPSHIRE,A,A
3,{8355F009-1A33-55C5-E053-6B04A8C0D090},182000,2003-12-12 00:00,BB3 2JS,S,N,L,MOORHURST,NaN,ASHLEIGH STREET,NaN,DARWEN,BLACKBURN WITH DARWEN,BLACKBURN WITH DARWEN,A,A
4,{8355F009-2140-55C5-E053-6B04A8C0D090},130000,2003-10-31 00:00,PE23 4JB,S,N,F,POST COTTAGE,NaN,NaN,LUSBY,SPILSBY,EAST LINDSEY,LINCOLNSHIRE,A,A


In [12]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [14]:
print(df_ppd.shape)
df_ppd.head()

(88053, 16)


,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
73722,{8355F009-351C-55C5-E053-6B04A8C0D090},1100000,2019-02-28,WA11 7QX,D,N,F,5,NaN,ST HELENS ROAD,RAINFORD,ST HELENS,ST HELENS,MERSEYSIDE,A,A
27599,{8355F009-AF69-55C5-E053-6B04A8C0D090},112000,2019-02-27,TS5 8RL,S,N,F,7,NaN,DENTON CLOSE,NaN,MIDDLESBROUGH,MIDDLESBROUGH,MIDDLESBROUGH,A,A
37794,{8355F009-5044-55C5-E053-6B04A8C0D090},168000,2019-02-27,TS9 5HF,S,N,F,45,NaN,ROSEBERRY AVENUE,STOKESLEY,MIDDLESBROUGH,HAMBLETON,NORTH YORKSHIRE,A,A
71292,{8355F009-8B70-55C5-E053-6B04A8C0D090},67000,2019-02-27,LL13 7HJ,F,N,L,130,NaN,STOCKWELL GROVE,NaN,WREXHAM,WREXHAM,WREXHAM,A,A
18106,{8355F009-CAE5-55C5-E053-6B04A8C0D090},175000,2019-02-27,LE13 1DJ,S,N,F,2,NaN,BURTON ROAD,NaN,MELTON MOWBRAY,MELTON,LEICESTERSHIRE,B,A


**Select rows that are from LONDON City only**

In [24]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()
streets

['HALLIWICK ROAD',
 'WELHAM ROAD',
 'WRICKLEMARSH ROAD',
 'CLEVELAND ROAD',
 'BRECKNOCK ROAD',
 'ST JOHNS AVENUE',
 'HIGH ROAD',
 'BODMIN STREET',
 'BRONDESBURY VILLAS',
 'LESSINGHAM AVENUE',
 'SHELDON ROAD',
 'CYRUS FIELD STREET',
 'CAVENDISH ROAD',
 'SHORT ROAD',
 'RADFORD ROAD',
 'BEECHDALE ROAD',
 'HOLLY BUSH ROW',
 'NEW CROSS ROAD',
 'MANOR GARDENS',
 'HERMITAGE ROAD',
 'WINSFORD TERRACE',
 'WILLESDEN LANE',
 'MORGAN AVENUE',
 'GLEDHOW GARDENS',
 'DORSET ROAD',
 'BELLE VUE ROAD',
 'HERTFORD AVENUE',
 'BOSCOMBE ROAD',
 'HILLMORE GROVE',
 'PUTNEY BRIDGE ROAD',
 'MABLETHORPE ROAD',
 'TALBOT ROAD',
 'NIMROD ROAD',
 'ROWAN CRESCENT',
 'STURGESS AVENUE',
 'FINCHLEY ROAD',
 'TELEGRAPH AVENUE',
 'NAVIGATION ROAD',
 'ESSEX WHARF',
 'MULREADY STREET',
 'WOLSELEY ROAD',
 'AMHURST PARK',
 'LORDSHIP ROAD',
 'KELMORE GROVE',
 'GRANDISON ROAD',
 'LOWESTOFT MEWS',
 'KINGSMERE PLACE',
 'STAMFORD HILL',
 'RANDOLPH AVENUE',
 'HAWKWOOD CRESCENT',
 'ST PAULS ROAD',
 'NAVARINO ROAD',
 'HOLMLEIGH ROAD',

**Find the average price of the property according to street wise**

In [16]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [23]:
df_grp_price.head(20)

,Street,Avg_Price
0,AARON HILL ROAD,295000.0
1,ABBERLEY MEWS,490000.0
2,ABBEY GARDENS,1802500.0
3,ABBEY PARADE,290000.0
4,ABBEY ROAD,663333.0
5,ABBEY STREET,750000.0
6,ABBOTS GARDENS,1310000.0
7,ABBOTS MANOR,562000.0
8,ABBOTSBURY CLOSE,1743500.0
9,ABBOTT AVENUE,355000.0


**You have to define your budget range here with max and min value of price with respect of df_grp_price dataframe**<br>
**Here we take a range of price which are in favourate street location**

In [19]:
df_affordable = df_grp_price.query("(Avg_Price >= 2000000) & (Avg_Price <= 3000000)")
#View your affordable dataset
df_affordable

,Street,Avg_Price
20,ACACIA ROAD,2.199975e+06
41,AIREDALE AVENUE,2.025000e+06
126,ARTHUR ROAD,2.800000e+06
237,BEDFORD ROW,2.450000e+06
276,BERKELEY STREET,3.000000e+06
308,BLACKFRIARS ROAD,2.147179e+06
330,BOLINGBROKE GROVE,2.275000e+06
395,BRITANNIA ROAD,2.057500e+06
403,BROAD WALK,2.065000e+06
420,BROMPTON ROAD,2.475000e+06


In [ ]:
london_data = pd.read_csv("Street Coordinates London.csv")

# Eliminate recurring word LONDON from individual names
london_data['Street'] = london_data['Street'].str.replace(', LONDON', '')

In [ ]:
df_pref_loc = pd.merge(london_data, df_affordable, on=['Street'], how='inner')

### Plot location of current market prices on map of LONDON

In [ ]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_pref_loc['Latitude'], df_pref_loc['Longitude'], df_pref_loc['Avg_Price'], df_pref_loc['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

**Define your foursquare credentials**

In [22]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '*******' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ******
CLIENT_SECRET:*******


### Export for preferred location for different venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Lets explore the first near neighborhood**

In [ ]:
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    
    facility_list=[]
    
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']

        # return only relevant information for each nearby venue
        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Street Name',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    return(nearby_facility)

In [ ]:
location_facility = getNearbyFacility(names=df_pref_loc['Street'],
                                      latitudes=df_pref_loc['Latitude'],
                                      longitudes=df_pref_loc['Longitude']
                                  )

### Finally over the assignment all results are saved in my final report section. Please review......